## Step 1 : Data Preparation

#### CONFIGURATIONS

In [ ]:
CLARITY_DATASET_SHC="<bigquery-project-id>:<source-dataset-shc>"
CLARITY_DATASET_LPCH="<bigquery-project-id>:<source-dataset-lpch>"

DATAPREP_DATASET_SHC="<bigquery-project-id>:<dataprep-dataset-shc>"
DATAPREP_DATASET_LPCH="<bigquery-project-id>:<dataprep-dataset-lpch>"

RESULT_DATASET_SHC="<bigquery-project-id>:<result-dataset-shc>"
RESULT_DATASET_LPCH="<bigquery-project-id>:<result-dataset-lpch>"

TARGET_GCS_BUCKET_SHC="<result-gcs-bucket-sch>"
TARGET_GCS_BUCKET_LPCH="<result-gcs-bucket-lpch>"

GOOGLE_CREDENTIAL="service-account-sample-key.json"

#DATAFLOW CONFIG
DATAFLOW_RUNNER_GCS_BUCKET="<gcs-bucket-for-staging-and-temp-files>"
DATAFLOW_PROJECT="<dataflow-project-id>"
DATAFLOW_DLP_PROJ="<dlp-project-id>"
DATAFLOW_SERVICEACCT="<service-account-email>"
DATAFLOW_MACHINE="n1-standard-8"
DATAFLOW_WORKER="200"

#RUN VERSION
RUN_VERSION="_v2"

#END OF CONFIGURATION
import subprocess
import os
import json
import sys
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(GOOGLE_CREDENTIAL)


In [ ]:
%%bash -s "$DATAFLOW_PROJECT"
gcloud config set project $1

#### BUILD PHI TABLE

In [ ]:
%%bash -s "$CLARITY_DATASET_SHC" "$CLARITY_DATASET_LPCH" "$DATAPREP_DATASET_SHC" "$DATAPREP_DATASET_LPCH" "$TARGET_GCS_BUCKET_SHC" "$TARGET_GCS_BUCKET_LPCH" "$DATAFLOW_RUNNER_GCS_BUCKET"
echo "bash bq__merge_phi.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./') $3.PHI_MERGED"
echo "bash bq__merge_phi.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./') $4.PHI_MERGED"

#or copy existing phi_merged
echo "bq cp som-rit-phi-starr-dev:rit_phi_clarity_deid_shc.PHI_MERGED $3.PHI_MERGED"
echo "bq cp som-rit-phi-starr-dev:rit_phi_clarity_deid_lpch.PHI_MERGED $4.PHI_MERGED"
#or copy existing EMERGENCY_CONTACTS
echo "bq cp som-rit-phi-starr-dev:rit_phi_clarity_deid_shc.CLARITY_EMP $3.CLARITY_EMP"
echo "bq cp som-rit-phi-starr-dev:rit_phi_clarity_deid_lpch.CLARITY_EMP $4.CLARITY_EMP"


#### MERGE LINES for NOTE, IMPRESSION, NARRATIVE

In [ ]:
%%bash -s "$CLARITY_DATASET_SHC" "$CLARITY_DATASET_LPCH" "$DATAPREP_DATASET_SHC" "$DATAPREP_DATASET_LPCH" "$TARGET_GCS_BUCKET_SHC" "$TARGET_GCS_BUCKET_LPCH" "$DATAFLOW_RUNNER_GCS_BUCKET"
echo "#### NOTE ###"
echo "bash bq__merge_note.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./') $3.HNO_NOTE_TEXT_MERGED"
echo "bash bq__merge_note.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./') $4.HNO_NOTE_TEXT_MERGED"
echo "#### IMPRESSION ###"
echo "bash bq__merge_impression.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./') $3.ORDER_IMPRESSION_MERGED"
echo "bash bq__merge_impression.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./') $4.ORDER_IMPRESSION_MERGED"
echo "#### NARRATIVE ###"
echo "bash bq__merge_narrative.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./') $3.ORDER_NARRATIVE_MERGED"
echo "bash bq__merge_narrative.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./') $4.ORDER_NARRATIVE_MERGED"



#### PREPARE DEID INPUT DATA for NOTE, ENCOUNTER,IMPRESSION,NARRATIVE

In [ ]:
%%bash -s "$CLARITY_DATASET_SHC" "$CLARITY_DATASET_LPCH" "$DATAPREP_DATASET_SHC" "$DATAPREP_DATASET_LPCH" "$TARGET_GCS_BUCKET_SHC" "$TARGET_GCS_BUCKET_LPCH" "$DATAFLOW_RUNNER_GCS_BUCKET"
echo "#### NOTE ###"
echo "bash bq_dataflow_prep_note.sh $(echo $3 | sed -e 's/[:]/\./')  $3.HNO_NOTE_TEXT_PHI_MERGED"
echo "bash bq_dataflow_prep_note.sh $(echo $4 | sed -e 's/[:]/\./')  $4.HNO_NOTE_TEXT_PHI_MERGED"
echo "#### ENCOUNTER ###"
echo "bash bq_dataflow_prep_encounter.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./')  $3.PAT_ENC_RSN_VISIT_PHI"
echo "bash bq_dataflow_prep_encounter.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./')  $4.PAT_ENC_RSN_VISIT_PHI"
echo "#### IMPRESSION ###"
echo "bash bq_dataflow_prep_impression.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./')  $3.ORDER_IMPRESSION_PHI_MERGED"
echo "bash bq_dataflow_prep_impression.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./')  $4.ORDER_IMPRESSION_PHI_MERGED"
echo "#### NARRATIVE ###"
echo "bash bq_dataflow_prep_narrative.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./')  $3.ORDER_NARRATIVE_PHI_MERGED"
echo "bash bq_dataflow_prep_narrative.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./')  $4.ORDER_NARRATIVE_PHI_MERGED"
echo "#### OR_LOG_ALL_PROC ###"
echo "bash bq_dataflow_prep_or_log_all_proc.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./')  $3.OR_LOG_ALL_PROC_PHI"
echo "bash bq_dataflow_prep_or_log_all_proc.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./')  $4.OR_LOG_ALL_PROC_PHI"
echo "#### IP_FLWSHT_MEAS ###"
echo "bash bq_dataflow_prep_flwsht_meas_comment.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./')  $3.IP_FLWSHT_MEAS_COMMENT_PHI"
echo "bash bq_dataflow_prep_flwsht_meas_comment.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./')  $4.IP_FLWSHT_MEAS_COMMENT_PHI"
echo "#### ORDER_RES_COMP_CMT ###"
echo "bash bq_dataflow_prep_order_res_comp_cmt.sh $(echo $1 | sed -e 's/[:]/\./') $(echo $3 | sed -e 's/[:]/\./')  $3.ORDER_RES_COMP_CMT_PHI"
echo "bash bq_dataflow_prep_order_res_comp_cmt.sh $(echo $2 | sed -e 's/[:]/\./') $(echo $4 | sed -e 's/[:]/\./')  $4.ORDER_RES_COMP_CMT_PHI"
echo "###END###"



#### EXPORT DEID INPUT DATA TO GCS 
Optional. Deid can also directly read from BigQuery

In [ ]:
%%bash -s "$CLARITY_DATASET_SHC" "$CLARITY_DATASET_LPCH" "$DATAPREP_DATASET_SHC" "$DATAPREP_DATASET_LPCH" "$TARGET_GCS_BUCKET_SHC" "$TARGET_GCS_BUCKET_LPCH" "$DATAFLOW_RUNNER_GCS_BUCKET" "$RUN_VERSION"
echo "#### NOTE ###"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$3.HNO_NOTE_TEXT_PHI_MERGED' $5/input$8/HNO_NOTE_TEXT_PHI_MERGED/note-input-*.json"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$4.HNO_NOTE_TEXT_PHI_MERGED' $6/input$8/HNO_NOTE_TEXT_PHI_MERGED/note-input-*.json"
echo "#### ENCOUNTER ###"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$3.PAT_ENC_RSN_VISIT_PHI' $5/input$8/PAT_ENC_RSN_VISIT_PHI/note-input-*.json"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$4.PAT_ENC_RSN_VISIT_PHI' $6/input$8/PAT_ENC_RSN_VISIT_PHI/note-input-*.json"
echo "#### IMPRESSION ###"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$3.ORDER_IMPRESSION_PHI_MERGED' $5/input$8/ORDER_IMPRESSION_PHI_MERGED/note-input-*.json"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$4.ORDER_IMPRESSION_PHI_MERGED' $6/input$8/ORDER_IMPRESSION_PHI_MERGED/note-input-*.json"
echo "#### NARRATIVE ###"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$3.ORDER_NARRATIVE_PHI_MERGED' $5/input$8/ORDER_NARRATIVE_PHI_MERGED/note-input-*.json"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$4.ORDER_NARRATIVE_PHI_MERGED' $6/input$8/ORDER_NARRATIVE_PHI_MERGED/note-input-*.json"
echo "#### OR_LOG ###"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$3.OR_LOG_ALL_PROC_PHI' $5/input$8/OR_LOG_ALL_PROC_PHI/note-input-*.json"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$4.OR_LOG_ALL_PROC_PHI' $6/input$8/OR_LOG_ALL_PROC_PHI/note-input-*.json"
echo "#### FLWSHT ###"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$3.IP_FLWSHT_MEAS_COMMENT_PHI' $5/input$8/IP_FLWSHT_MEAS_COMMENT_PHI/note-input-*.json"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$4.IP_FLWSHT_MEAS_COMMENT_PHI' $6/input$8/IP_FLWSHT_MEAS_COMMENT_PHI/note-input-*.json"
echo "#### ORDER_RES_COMP_CMT ###"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$3.ORDER_RES_COMP_CMT_PHI' $5/input$8/ORDER_RES_COMP_CMT_PHI/note-input-*.json"
echo "bq --location=US extract --destination_format NEWLINE_DELIMITED_JSON '$4.ORDER_RES_COMP_CMT_PHI' $6/input$8/ORDER_RES_COMP_CMT_PHI/note-input-*.json"
echo "###END###"

## Step 2 : Deid Process
### run deid on DataFlow

In [ ]:
%%bash -s "$TARGET_GCS_BUCKET_SHC" "$TARGET_GCS_BUCKET_LPCH" "$DATAFLOW_RUNNER_GCS_BUCKET" "$DATAFLOW_PROJECT" "$DATAFLOW_DLP_PROJ" "$DATAFLOW_SERVICEACCT" "$DATAFLOW_MACHINE" "$DATAFLOW_WORKER"  "$RUN_VERSION"
cd ..
echo "export GOOGLE_APPLICATION_CREDENTIALS=/Users/wenchengli/dev/servers/gcp/som-rit-phi-starr-dev/som-rit-phi-starr-dev-abb03b2ca689.json"
CMDPRE="mvn -Pdataflow-runner compile exec:java -Dexec.mainClass=com.github.susom.starr.deid.Main -Dexec.args=\"--project=$4 --dlpProject=$5 --serviceAccount=$6 --stagingLocation=$3/staging --gcpTempLocation=$3/temp --tempLocation=$3/temp --region=us-west1 --workerMachineType=$7 --maxNumWorkers=$8 --diskSizeGb=100 --runner=DataflowRunner --deidConfigFile=deid_config_clarity.yaml --inputType=gcp_gcs  "
echo "#### NOTE ####"
echo "$CMDPRE --textIdFields=\"note_id,note_csn_id\" --textInputFields=\"fullnote\" --inputResource=$1/input_v2/HNO_NOTE_TEXT_PHI_MERGED/note-input-*.json --outputResource=$1/NOTE_DEID_result$9\""
echo "$CMDPRE --textIdFields=\"note_id,note_csn_id\" --textInputFields=\"fullnote\" --inputResource=$2/input_v2/HNO_NOTE_TEXT_PHI_MERGED/note-input-*.json --outputResource=$2/NOTE_DEID_result$9\""
echo "#### PAT_ENC_RSN_VISIT(ENCOUNTER) ####"
echo "$CMDPRE --textIdFields=\"PAT_ENC_CSN_ID,LINE\" --textInputFields=\"COMMENTS\" --inputResource=$1/input_v2/PAT_ENC_RSN_VISIT_PHI/note-input-*.json --outputResource=$1/PAT_ENC_RSN_VISIT_DEID_result$9\""
echo "$CMDPRE --textIdFields=\"PAT_ENC_CSN_ID,LINE\" --textInputFields=\"COMMENTS\" --inputResource=$2/input_v2/PAT_ENC_RSN_VISIT_PHI/note-input-*.json --outputResource=$2/PAT_ENC_RSN_VISIT_DEID_result$9\""
echo "#### ORDER_IMPRESSION ####"
echo "$CMDPRE --textIdFields=\"ORDER_PROC_ID\" --textInputFields=\"FULL_IMPRESSION\" --inputResource=$1/input_v2/ORDER_IMPRESSION_PHI_MERGED/note-input-*.json --outputResource=$1/ORDER_IMPRESSION_DEID_result$9\""
echo "$CMDPRE --textIdFields=\"ORDER_PROC_ID\" --textInputFields=\"FULL_IMPRESSION\" --inputResource=$2/input_v2/ORDER_IMPRESSION_PHI_MERGED/note-input-*.json --outputResource=$2/ORDER_IMPRESSION_DEID_result$9\""
echo "#### ORDER_NARRATIVE ####"
echo "$CMDPRE --textIdFields=\"ORDER_PROC_ID\" --textInputFields=\"FULL_NARRATIVE\" --inputResource=$1/input_v2/ORDER_NARRATIVE_PHI_MERGED/note-input-*.json --outputResource=$1/ORDER_NARRATIVE_DEID_result$9\""
echo "$CMDPRE --textIdFields=\"ORDER_PROC_ID\" --textInputFields=\"FULL_NARRATIVE\" --inputResource=$2/input_v2/ORDER_NARRATIVE_PHI_MERGED/note-input-*.json --outputResource=$2/ORDER_NARRATIVE_DEID_result$9\""
echo "#### OR_LOG ####"
echo "$CMDPRE --textIdFields=\"LOG_ID,LINE\" --textInputFields=\"COMMENTS\" --inputResource=$1/input_v2/OR_LOG_ALL_PROC_PHI/note-input-*.json --outputResource=$1/OR_LOG_ALL_PROC_DEID_result$9\""
echo "$CMDPRE --textIdFields=\"LOG_ID,LINE\" --textInputFields=\"COMMENTS\" --inputResource=$2/input_v2/OR_LOG_ALL_PROC_PHI/note-input-*.json --outputResource=$2/OR_LOG_ALL_PROC_DEID_result$9\""
echo "#### FLWSHT ####"
echo "$CMDPRE --textIdFields=\"FSD_ID,LINE\" --textInputFields=\"MEAS_VALUE,meas_comment\" --inputResource=$1/input_v2/IP_FLWSHT_MEAS_COMMENT_PHI/note-input-*.json --outputResource=$1/IP_FLWSHT_MEAS_DEID_result$9\""
echo "$CMDPRE --textIdFields=\"FSD_ID,LINE\" --textInputFields=\"MEAS_VALUE,meas_comment\" --inputResource=$2/input_v2/IP_FLWSHT_MEAS_COMMENT_PHI/note-input-*.json --outputResource=$2/IP_FLWSHT_MEAS_DEID_result$9\""
echo "#### ORDER_RES_COMP_CMT ####"
echo "$CMDPRE --textIdFields=\"ORDER_ID,CONTACT_DATE_REAL,LINE_COMP,LINE_COMMENT\" --textInputFields=\"RESULTS_COMP_CMT\" --inputResource=$1/input_v2/ORDER_RES_COMP_CMT_PHI/note-input-*.json --outputResource=$1/ORDER_RES_COMP_CMT_DEID_result$9\""
echo "$CMDPRE --textIdFields=\"ORDER_ID,CONTACT_DATE_REAL,LINE_COMP,LINE_COMMENT\" --textInputFields=\"RESULTS_COMP_CMT\" --inputResource=$2/input_v2/ORDER_RES_COMP_CMT_PHI/note-input-*.json --outputResource=$2/ORDER_RES_COMP_CMT_DEID_result$9\""
echo "###END###"

Run as jar

In [ ]:
%%bash -s "$TARGET_GCS_BUCKET_SHC" "$TARGET_GCS_BUCKET_LPCH" "$DATAFLOW_RUNNER_GCS_BUCKET" "$DATAFLOW_PROJECT" "$DATAFLOW_DLP_PROJ" "$DATAFLOW_SERVICEACCT" "$DATAFLOW_MACHINE" "$DATAFLOW_WORKER"  "$RUN_VERSION"
cd ..
CMDPRE="java -jar /Users/wenchengli/dev/github/starr-core/deid/target/deid-2.0.jar --project=$4 --dlpProject=$5 --serviceAccount=$6 --stagingLocation=$3/staging --gcpTempLocation=$3/temp --tempLocation=$3/temp --region=us-west1 --workerMachineType=$7 --maxNumWorkers=$8 --diskSizeGb=100 --runner=DataflowRunner --deidConfigFile=deid_config_clarity.yaml --inputType=gcp_gcs  "
echo "#### NOTE ####"
echo "$CMDPRE --textIdFields=\"note_id,note_csn_id\" --textInputFields=\"fullnote\" --inputResource=$1/input_v2/HNO_NOTE_TEXT_PHI_MERGED/note-input-*.json --outputResource=$1/NOTE_DEID_result$9"
echo "$CMDPRE --textIdFields=\"note_id,note_csn_id\" --textInputFields=\"fullnote\" --inputResource=$2/input_v2/HNO_NOTE_TEXT_PHI_MERGED/note-input-*.json --outputResource=$2/NOTE_DEID_result$9"
echo "#### PAT_ENC_RSN_VISIT(ENCOUNTER) ####"
echo "$CMDPRE --textIdFields=\"PAT_ENC_CSN_ID,LINE\" --textInputFields=\"COMMENTS\" --inputResource=$1/input_v2/PAT_ENC_RSN_VISIT_PHI/note-input-*.json --outputResource=$1/PAT_ENC_RSN_VISIT_DEID_result$9"
echo "$CMDPRE --textIdFields=\"PAT_ENC_CSN_ID,LINE\" --textInputFields=\"COMMENTS\" --inputResource=$2/input_v2/PAT_ENC_RSN_VISIT_PHI/note-input-*.json --outputResource=$2/PAT_ENC_RSN_VISIT_DEID_result$9"
echo "#### ORDER_IMPRESSION ####"
echo "$CMDPRE --textIdFields=\"ORDER_PROC_ID\" --textInputFields=\"FULL_IMPRESSION\" --inputResource=$1/input_v2/ORDER_IMPRESSION_PHI_MERGED/note-input-*.json --outputResource=$1/ORDER_IMPRESSION_DEID_result$9"
echo "$CMDPRE --textIdFields=\"ORDER_PROC_ID\" --textInputFields=\"FULL_IMPRESSION\" --inputResource=$2/input_v2/ORDER_IMPRESSION_PHI_MERGED/note-input-*.json --outputResource=$2/ORDER_IMPRESSION_DEID_result$9"
echo "#### ORDER_NARRATIVE ####"
echo "$CMDPRE --textIdFields=\"ORDER_PROC_ID\" --textInputFields=\"FULL_NARRATIVE\" --inputResource=$1/input_v2/ORDER_NARRATIVE_PHI_MERGED/note-input-*.json --outputResource=$1/ORDER_NARRATIVE_DEID_result$9"
echo "$CMDPRE --textIdFields=\"ORDER_PROC_ID\" --textInputFields=\"FULL_NARRATIVE\" --inputResource=$2/input_v2/ORDER_NARRATIVE_PHI_MERGED/note-input-*.json --outputResource=$2/ORDER_NARRATIVE_DEID_result$9"
echo "#### OR_LOG ####"
echo "$CMDPRE --textIdFields=\"LOG_ID,LINE\" --textInputFields=\"COMMENTS\" --inputResource=$1/input_v2/OR_LOG_ALL_PROC_PHI/note-input-*.json --outputResource=$1/OR_LOG_ALL_PROC_DEID_result$9"
echo "$CMDPRE --textIdFields=\"LOG_ID,LINE\" --textInputFields=\"COMMENTS\" --inputResource=$2/input_v2/OR_LOG_ALL_PROC_PHI/note-input-*.json --outputResource=$2/OR_LOG_ALL_PROC_DEID_result$9"
echo "#### FLWSHT ####"
echo "$CMDPRE --textIdFields=\"FSD_ID,LINE\" --textInputFields=\"MEAS_VALUE,meas_comment\" --inputResource=$1/input_v2/IP_FLWSHT_MEAS_COMMENT_PHI/note-input-*.json --outputResource=$1/IP_FLWSHT_MEAS_DEID_result$9"
echo "$CMDPRE --textIdFields=\"FSD_ID,LINE\" --textInputFields=\"MEAS_VALUE,meas_comment\" --inputResource=$2/input_v2/IP_FLWSHT_MEAS_COMMENT_PHI/note-input-*.json --outputResource=$2/IP_FLWSHT_MEAS_DEID_result$9"
echo "#### ORDER_RES_COMP_CMT ####"
echo "$CMDPRE --textIdFields=\"ORDER_ID,CONTACT_DATE_REAL,LINE_COMP,LINE_COMMENT\" --textInputFields=\"RESULTS_COMP_CMT\" --inputResource=$1/input_v2/ORDER_RES_COMP_CMT_PHI/note-input-*.json --outputResource=$1/ORDER_RES_COMP_CMT_DEID_result$9"
echo "$CMDPRE --textIdFields=\"ORDER_ID,CONTACT_DATE_REAL,LINE_COMP,LINE_COMMENT\" --textInputFields=\"RESULTS_COMP_CMT\" --inputResource=$2/input_v2/ORDER_RES_COMP_CMT_PHI/note-input-*.json --outputResource=$2/ORDER_RES_COMP_CMT_DEID_result$9"
echo "#### DONE ####"

## Step 3 : Load Result 
### load result to BigQuery

In [ ]:
%%bash -s "$TARGET_GCS_BUCKET_SHC" "$TARGET_GCS_BUCKET_LPCH" "$RESULT_DATASET_SHC" "$RESULT_DATASET_LPCH"  "$RUN_VERSION"
cd ..
CMDPRE="bq --location=US load --autodetect  --source_format=NEWLINE_DELIMITED_JSON "
echo "#### NOTE ####"
echo "$CMDPRE $3.HNO_NOTE_TEXT_MERGED_DEIDED" $1/NOTE_DEID_result$5/DeidNote* 
echo "$CMDPRE $4.HNO_NOTE_TEXT_MERGED_DEIDED" $2/NOTE_DEID_result$5/DeidNote*
echo "#### PAT_ENC_RSN_VISIT(ENCOUNTER) ####"
echo "$CMDPRE $3.PAT_ENC_RSN_VISIT_DEID" $1/PAT_ENC_RSN_VISIT_DEID_result$5/DeidNote*
echo "$CMDPRE $4.PAT_ENC_RSN_VISIT_DEID" $2/PAT_ENC_RSN_VISIT_DEID_result$5/DeidNote* 
echo "#### ORDER_IMPRESSION ####"
echo "$CMDPRE $3.ORDER_IMPRESSION_DEID" $1/ORDER_IMPRESSION_DEID_result$5/DeidNote*
echo "$CMDPRE $4.ORDER_IMPRESSION_DEID" $2/ORDER_IMPRESSION_DEID_result$5/DeidNote*
echo "#### ORDER_NARRATIVE ####"
echo "$CMDPRE $3.ORDER_NARRATIVE_DEID" $1/ORDER_NARRATIVE_DEID_result$5/DeidNote*
echo "$CMDPRE $4.ORDER_NARRATIVE_DEID" $2/ORDER_NARRATIVE_DEID_result$5/DeidNote*
echo "#### OR_LOG ####"
echo "$CMDPRE $3.OR_LOG_ALL_PROC_DEID" $1/OR_LOG_ALL_PROC_DEID_result$5/DeidNote*
echo "$CMDPRE $4.OR_LOG_ALL_PROC_DEID" $2/OR_LOG_ALL_PROC_DEID_result$5/DeidNote*
echo "#### FLWSHT ####"
echo "$CMDPRE $3.IP_FLWSHT_MEAS_DEID" $1/IP_FLWSHT_MEAS_DEID_result$5/DeidNote*
echo "$CMDPRE $4.IP_FLWSHT_MEAS_DEID" $2/IP_FLWSHT_MEAS_DEID_result$5/DeidNote*
echo "#### ORDER_RES_COMP_CMT ####"
echo "$CMDPRE $3.ORDER_RES_COMP_CMT_DEID" $1/ORDER_RES_COMP_CMT_DEID_result$5/DeidNote*
echo "$CMDPRE $4.ORDER_RES_COMP_CMT_DEID" $2/ORDER_RES_COMP_CMT_DEID_result$5/DeidNote*
echo "###END###"

### apply confidencial filter for note

In [ ]:
%%bash -s "$TARGET_GCS_BUCKET_SHC" "$TARGET_GCS_BUCKET_LPCH" "$RESULT_DATASET_SHC" "$RESULT_DATASET_LPCH"  "$RUN_VERSION"
echo "#### NOTE CONFIDENCIAL FILTER ###"
echo "bash bq_final_confidencial_filter_note.sh $(echo $3 | sed -e 's/[:]/\./').HNO_NOTE_TEXT_MERGED_DEIDED  $3.HNO_NOTE_TEXT_MERGED_DEIDED_FILTERED"
echo "bash bq_final_confidencial_filter_note.sh $(echo $4 | sed -e 's/[:]/\./').HNO_NOTE_TEXT_MERGED_DEIDED  $4.HNO_NOTE_TEXT_MERGED_DEIDED_FILTERED"